In [1]:
!pip install transformers lime hf_xet datasets captum numpy

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader
#from transformers import AutoTokenizer, AutoModelForCausalLM
from lime.lime_text import LimeTextExplainer
from datasets import load_dataset
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
import warnings
import tqdm as tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig
from captum.attr import IntegratedGradients
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
warnings.filterwarnings("ignore")


In [3]:
dataset = load_dataset("hatexplain")

In [4]:
dataset['train'][0]

{'id': '23107796_gab',
 'annotators': {'label': [0, 2, 2],
  'annotator_id': [203, 204, 233],
  'target': [['Hindu', 'Islam'],
   ['Hindu', 'Islam'],
   ['Hindu', 'Islam', 'Other']]},
 'rationales': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]],
 'post_tokens': ['u',
  'really',
  'think',
  'i',
  'would',
  'not',
  'have',
  'been',
  'raped',
  'by',
  'feral',
  'hindu',
  'or',
  'muslim',
  'back',
  'in',
  'india',
  'or',
  'bangladesh',
  'and',
  'a',
  'neo',
  'nazi',
  'would',
  'rape',
  'me',
  'as',
  'well',
  'just',
  'to',
  'see',
  'me',
  'cry']}

In [5]:
def join_tokens(example):
    example["text"] = " ".join(example["post_tokens"])
    return example

dataset = dataset.map(join_tokens)

dataset["train"][0]["text"]

'u really think i would not have been raped by feral hindu or muslim back in india or bangladesh and a neo nazi would rape me as well just to see me cry'

In [6]:
def combine_rationale_masks(rationale_masks, method='weighted'):
    """
    Combines multiple binary rationale masks into a single mask.
    """
    rationale_array = np.array(rationale_masks)

    # Union method: If any annotator highlights a token, it is part of the rationale
    if method == 'union':
        final_rationale_mask = (rationale_array.sum(axis=0) > 0).astype(int)

    # Majority method: A token is part of the rationale if the majority of annotators highlight it
    elif method == 'majority':
        final_rationale_mask = (rationale_array.sum(axis=0) >= (len(rationale_masks) / 2)).astype(int)

    # Weighted method: Tokens are weighted based on how many annotators highlight it
    elif method == 'weighted':
        # Compute the average weight for each token (0-1 scale)
        weights = rationale_array.mean(axis=0)
        # Map average values to 0 or 1, with values < 1 being fractional
        final_rationale_mask = weights.tolist()

    else:
        raise ValueError("Unknown method. Choose from 'union', 'majority', or 'weighted'.")

    return final_rationale_mask

def safe_add_final_rationale(example):
    rationales = example.get('rationales', None)

    if not rationales or not isinstance(rationales, list) or not all(isinstance(r, list) for r in rationales):
        # Default to a zero mask if invalid or missing
        num_tokens = len(example.get('post_tokens', []))
        example['final_rationale_mask'] = [0] * num_tokens
        return example

    try:
        example['final_rationale_mask'] = combine_rationale_masks(rationales, method='weighted')
    except Exception as e:
        # Handle edge cases or shape mismatch
        num_tokens = len(example.get('post_tokens', []))
        example['final_rationale_mask'] = [0] * num_tokens

    return example

dataset = dataset.map(safe_add_final_rationale)

In [7]:
dataset['train'][17]

{'id': '1177990839177490432_twitter',
 'annotators': {'label': [0, 0, 0],
  'annotator_id': [6, 25, 10],
  'target': [['African'],
   ['Disability', 'African', 'Homosexual'],
   ['Women', 'Disability', 'African', 'Homosexual']]},
 'rationales': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]],
 'post_tokens': ['nigger',
  'faggot',
  'retard',
  'beaner',
  'dishwasher',
  'time',
  'to',
  'see',
  'if',
  'i',
  'get',
  'ban'],
 'text': 'nigger faggot retard beaner dishwasher time to see if i get ban',
 'final_rationale_mask': [1.0,
  1.0,
  1.0,
  1.0,
  0.3333333333333333,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]}

In [8]:
from collections import Counter

def resolve_hate_label(label_list):
    """
    Convert multi-class label (0: hate, 1: neither, 2: offensive) to binary label.
    Returns 'hate' if majority is 0 or 2; otherwise 'non-hate'.
    """
    if not label_list or len(label_list) != 3:
        return "non-detrimental"  # fallback
    counts = Counter(label_list)
    majority_label = counts.most_common(1)[0][0]
    return "detrimenal_content" if majority_label in [0, 2] else "non-detrimental"
def add_binary_label(example):
    example['binary_label'] = resolve_hate_label(example['annotators']['label'])
    return example

dataset = dataset.map(add_binary_label)

In [9]:
dataset['train'][17]

{'id': '1177990839177490432_twitter',
 'annotators': {'label': [0, 0, 0],
  'annotator_id': [6, 25, 10],
  'target': [['African'],
   ['Disability', 'African', 'Homosexual'],
   ['Women', 'Disability', 'African', 'Homosexual']]},
 'rationales': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]],
 'post_tokens': ['nigger',
  'faggot',
  'retard',
  'beaner',
  'dishwasher',
  'time',
  'to',
  'see',
  'if',
  'i',
  'get',
  'ban'],
 'text': 'nigger faggot retard beaner dishwasher time to see if i get ban',
 'final_rationale_mask': [1.0,
  1.0,
  1.0,
  1.0,
  0.3333333333333333,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'binary_label': 'detrimenal_content'}

In [10]:
from torch.utils.data import Dataset

class HateXplainDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=128):
        """
        dataset: HuggingFace-style Dataset object (already mapped with final_rationale_mask)
        tokenizer: Tokenizer (e.g., from transformers) to tokenize 'text'
        max_length: Max token length for model inputs
        """
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_map = {"non_detrimenal": 0, "detrimental": 1}  # adjust if needed

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]

        # Tokenize text
        encoding = self.tokenizer(
            item['text'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Convert label to binary
        label = 1 if item['binary_label'] == 'detrimenal_content' else 0

        # Final rationale mask (assumes already aligned with post_tokens)
        rationale = item.get('final_rationale_mask', [])

        # Pad rationale to max_length
        rationale_tensor = torch.zeros(self.max_length)
        rationale_values = rationale[:self.max_length]
        rationale_tensor[:len(rationale_values)] = torch.tensor(rationale_values, dtype=torch.float32)

        return {
            'input_ids': encoding['input_ids'].squeeze(0),  # [max_length]
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.float32),
            'rationale_mask': rationale_tensor
        }

In [11]:
import torch
import torch.nn as nn
from transformers import RobertaModel, RobertaConfig

class CustomRobertaClassifier(nn.Module):
    def __init__(self, pretrained_model_name, hidden_size=128, dropout_rate=0.3):
        super(CustomRobertaClassifier, self).__init__()

        self.roberta = RobertaModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(dropout_rate)

        # Intermediate dense layer
        self.dense = nn.Linear(self.roberta.config.hidden_size, hidden_size)
        self.relu = nn.ReLU()

        # Final classification layer (outputting 1 value for binary classification)
        self.classifier = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [batch_size, hidden_size]

        x = self.dropout(pooled_output)
        x = self.relu(self.dense(x))
        x = self.dropout(x)
        logits = self.classifier(x)

        return self.sigmoid(logits)





In [12]:


# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


model = CustomRobertaClassifier(pretrained_model_name='roberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
train_dataset = HateXplainDataset(dataset['train'], tokenizer)
validation_dataset = HateXplainDataset(dataset['validation'], tokenizer)
test_dataset = HateXplainDataset(dataset['test'], tokenizer)

In [14]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [15]:
def evaluate(model, dataloader, device='cuda'):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            preds = (outputs.squeeze() > 0.5).int()

            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().int().tolist())

    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    print(f"Validation Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")
    return f1

In [16]:
from captum.attr import LayerIntegratedGradients
import math
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=['0', '1'])

def predict_proba(texts):
    model.eval()
    all_probs = []
    for text in texts:
        enc = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
        with torch.no_grad():
            probs = model(enc['input_ids'], enc['attention_mask'])
        probs = torch.cat([1 - probs, probs], dim=1)  # shape: (1, 2)
        all_probs.append(probs.cpu().numpy()[0])
    return np.array(all_probs)


    return out
def train(model, train_loader, val_loader, epochs=3, lr=2e-5, lambda_reg=0.5, device='cuda'):
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        prev_val_f1 = 0.5
        loop =tqdm(train_loader, desc=f"Epoch {epoch+1} Training")
        for batch in loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].unsqueeze(1).to(device)
            rationale_mask = batch['rationale_mask'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)

            pred_loss = criterion(outputs, labels)
            if math.isnan(pred_loss.item()):
              print('NAAAAN')
            # Integrated Gradients (token-level importance)


            for i in range(len(labels)):
                if labels[i].item() == 1:
                    model.zero_grad()

                    # Get raw text from input_ids
                    tokens = tokenizer.convert_ids_to_tokens(input_ids[i])
                    raw_text = tokenizer.decode(input_ids[i], skip_special_tokens=True)

                    # Use LIME to get token importance
                    explanation = explainer.explain_instance(raw_text, predict_proba, num_features=len(tokens), labels=[1], num_samples = 32)
                    lime_weights = dict(explanation.as_list(label=1))

                    # Map LIME weights to token indices
                    input_tokens = tokenizer.tokenize(raw_text)
                    lime_vector = torch.zeros_like(input_ids[i], dtype=torch.float)

                    for j, token_id in enumerate(input_ids[i]):
                        token = tokenizer.convert_ids_to_tokens(token_id.item())
                        for lime_token, score in lime_weights.items():
                            if lime_token in token:
                                lime_vector[j] = max(lime_vector[j], score)

                    lime_vector = lime_vector.clamp(min=0)
                    norm_lime = lime_vector / (lime_vector.sum() + 1e-8)

                    if rationale_mask[i].sum().item() == 0:
                        continue

                    target_rationale = rationale_mask[i].float() / (rationale_mask[i].sum() + 1e-8)
                    target_rationale = target_rationale + 1e-8

                    kl_div = (target_rationale * (target_rationale / (norm_lime + 1e-8)).log()).sum()
                    pred_loss += lambda_reg * kl_div






            pred_loss.backward()
            optimizer.step()

            total_loss += pred_loss.item()
            loop.set_postfix(loss=pred_loss / 8)
        print(f"Epoch {epoch+1} Training Loss: {total_loss / len(train_loader):.4f}")
        f1 = evaluate(model, val_loader, device)
        if f1 > prev_val_f1:
          prev_val_f1 = f1
          torch.save(model.state_dict(), f'hatexplain_roberta_weights_LIME_{round(f1,4)}.pth')

In [17]:
lambs = [5, 10]
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
for lamb in lambs:
  print('LAMBDA: ',lamb)
  model = CustomRobertaClassifier(pretrained_model_name='roberta-base')
  model.to(device)
  train(model, train_loader, val_loader, epochs=3, lr=2e-5, lambda_reg=lamb)


LAMBDA:  5


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1 Training: 100%|██████████| 1923/1923 [53:20<00:00,  1.66s/it, loss=tensor(6.0824, device='cuda:0', grad_fn=<DivBackward0>)]


Epoch 1 Training Loss: 208.3325


Evaluating: 100%|██████████| 241/241 [00:13<00:00, 18.22it/s]


Validation Accuracy: 0.7643, F1 Score: 0.8105


...

Validation Accuracy: 0.7882, F1 Score: 0.8195


Epoch 3 Training: 100%|██████████| 1923/1923 [52:40<00:00,  1.64s/it, loss=tensor(32.6552, device='cuda:0', grad_fn=<DivBackward0>)]


Epoch 3 Training Loss: 407.7020


Evaluating: 100%|██████████| 241/241 [00:13<00:00, 18.23it/s]


Validation Accuracy: 0.7794, F1 Score: 0.8132


In [18]:
torch.save(model.state_dict(), 'hatexplain_roberta_weights_LIME.pth')

In [19]:
!zip -r files_LIME.zip . -x "*.ipynb_checkpoints*"

  adding: .config/ (stored 0%)
  adding: .config/.last_update_check.json (deflated 24%)
  adding: .config/configurations/ (stored 0%)
  adding: .config/configurations/config_default (deflated 15%)
  adding: .config/logs/ (stored 0%)
  adding: .config/logs/2025.04.28/ (stored 0%)
  adding: .config/logs/2025.04.28/13.34.26.846099.log (deflated 58%)
  adding: .config/logs/2025.04.28/13.33.57.096159.log (deflated 92%)
  adding: .config/logs/2025.04.28/13.34.35.168196.log (deflated 86%)
  adding: .config/logs/2025.04.28/13.34.50.274199.log (deflated 56%)
  adding: .config/logs/2025.04.28/13.34.41.209709.log (deflated 58%)
  adding: .config/logs/2025.04.28/13.34.49.603666.log (deflated 57%)
  adding: .config/default_configs.db (deflated 98%)
  adding: .config/gce (stored 0%)
  adding: .config/.last_opt_in_prompt.yaml (stored 0%)
  adding: .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
  adding: .config/config_sentinel (stored 0%)
  adding: .config/activ

In [20]:
from google.colab import files
files.download("files.zip")

FileNotFoundError: Cannot find file: files.zip